In [ ]:
#coding:utf-8

######模拟登陆######
import requests
import urllib
import base64
import time
import re
import json
import rsa
import binascii
from bs4 import BeautifulSoup
from requests.packages.urllib3.connectionpool import InsecureRequestWarning
import pandas as pd
import numpy as np
import winsound

header = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.22 Safari/537.36',
        'Accept-Encoding': 'gzip, deflate',
        'Accept-Language': 'zh-CN,zh;q=0.8',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Connection': 'keep-alive',
        'Cache-Control': 'max-age=0'
        }

class Login(object):
    
    session = requests.session()
    user_name = '' #填入微博账号
    pass_word = '' #填入微博密码
        
    def get_username(self):
        return base64.b64encode(urllib.parse.quote(self.user_name).encode('utf-8')).decode('utf-8')
    
    def get_pre_login(self):
        params = {
            'entry': 'weibo',
            'callback': 'sinaSSOController.preloginCallBack',
            'su': self.get_username(),
            'rsakt': 'mod',
            'client': 'ssologin.js(v1.4.19)',
            '_': int(time.time() * 1000)
            }
        try:
            response = self.session.post('https://login.sina.com.cn/sso/prelogin.php',params = params, headers = header, verify = False)
            return json.loads(re.search(r'\((?P<data>.*)\)', response.text).group('data'))
        except:
            print('获取公钥失败')
            return 0
        
    def get_password(self):
        #dynamic = self.get_pre_login()
        public_key = rsa.PublicKey(int(dynamic['pubkey'],16),int('10001',16))
        password_string = str(dynamic['servertime']) + '\t' + str(dynamic['nonce']) + '\n' + self.pass_word
        return binascii.b2a_hex(rsa.encrypt(password_string.encode('utf-8'),public_key)).decode('utf-8')
    
    def login(self):
        global dynamic
        dynamic = self.get_pre_login()
        post_data = {
                    'entry': 'weibo',
                    'gateway': '1',
                    'from': '',
                    'savestate': '7',
                    'qrcode_flag': 'false',
                    'useticket': '1',
                    'vsnf': '1',
                    'su': self.get_username(),
                    'service': 'miniblog',
                    'servertime': dynamic['servertime'],
                    'nonce': dynamic['nonce'],
                    'pwencode': 'rsa2',
                    'rsakv': dynamic['rsakv'],
                    'sp': self.get_password(),
                    'sr': '1366*768',
                    'encoding': 'UTF-8',
                    'prelt': '212',
                    'url': 'https://weibo.com/ajaxlogin.php?framelogin=1&callback=parent.sinaSSOController.feedBackUrlCallBack',
                    'returntype': 'TEXT'
                    }
        login_data = self.session.post('https://login.sina.com.cn/sso/login.php?client=ssologin.js(v1.4.19)', data = post_data, headers = header, verify = False)

        params = {
                'ticket': login_data.json()['ticket'],
                'ssosavestate': int(time.time()),
                'callback': 'sinaSSOController.doCrossDomainCallBack',
                'scriptId': 'ssoscript0',
                'client': 'ssologin.js(v1.4.19)',
                '_': int(time.time() * 1000)
                }
        self.session.post('https://passport.weibo.com/wbsso/login',params = params, headers = header, verify = False)
        return self.session
    
login = Login()
session = login.login()
'''
response = session.post('https://weibo.com/', headers = header, verify = False)
soup = BeautifulSoup(response.text, 'html.parser')
print(soup.find('title'))
'''

######数据抓取######
def get_page_session(date):
    time.sleep(8)
    return session.post('https://s.weibo.com/weibo?q=%E8%BD%AC%E5%9F%BA%E5%9B%A0%20-%E6%B2%AA%E6%B7%B1%20-%E8%82%A1%E5%B8%82%20-%E6%9D%BF%E5%9D%97%20-%E6%8C%87%E6%95%B0%20-%E5%85%8D%E8%B4%B9%E5%9B%B4%E8%A7%82%20-%E5%BF%AB%E6%9D%A5%E4%B8%80%E8%B5%B7%E5%9B%B4%E8%A7%82&scope=ori&suball=1&timescope=custom:2014-08-31-0:2014-09-01-0&Refer=g&page='+str(page), headers = header, verify = False)

def get_data_session(date, page):
    time.sleep(8)
    return session.post('https://s.weibo.com/weibo?q=%E8%BD%AC%E5%9F%BA%E5%9B%A0%20-%E6%B2%AA%E6%B7%B1%20-%E8%82%A1%E5%B8%82%20-%E6%9D%BF%E5%9D%97%20-%E6%8C%87%E6%95%B0%20-%E5%85%8D%E8%B4%B9%E5%9B%B4%E8%A7%82%20-%E5%BF%AB%E6%9D%A5%E4%B8%80%E8%B5%B7%E5%9B%B4%E8%A7%82&scope=ori&suball=1&timescope=custom:2014-08-31-0:2014-09-01-0&Refer=g&page='+str(page), headers = header, verify = False)

def get_page_res(date):
    try:
        return get_page_session(date)
    except:
        try:
            return get_page_session(date)
        except:
            print('获取页码信息失败', date)
            return 0
        
def get_data_res(date, page):
    try:
        return get_data_session(date, page)
    except:
        try:
            return get_data_session(date, page)
        except:
            print('获取页码信息失败', date, page)
            return 0

def get_page(date):
    response = get_page_res(date)
    if response:
        try:
            soup = BeautifulSoup(response.text, 'html.parser')
            pages = soup.find('ul', 's-scroll').find_all('li')
            return len(pages)
        except:
            print('获取页码失败', date)
            return 0

def get_data(date, page):
    response = get_data_res(date, page)
    data = []
    if response:
        try:
            soup = BeautifulSoup(response.text, 'html.parser')
            infos = soup.find_all('div', 'content')
            records = soup.find_all('div', 'card-act')
            for info, record in zip(infos[0:], records[0:]):
                #日期、时间
                times = ''.join(info.find('p', 'from').text)
                data.append(''.join(times.split()[0]))
                data.append(''.join(times.split()[1]))
                
                #用户名
                user = info.find('a', 'name')
                data.append(''.join(user.text))
                
                #用户ID
                user_id = (re.search('href="//weibo.com/(.*?)?refer', str(info.find('a', 'name')))).group(1).rstrip('?')
                data.append(''.join(user_id))
                
                #博文ID
                rec = record.find_all('li')
                m_id = (re.search('mid=(.*?)&amp', str(rec[1]))).group(1)
                data.append(''.join(m_id))
                
                #用户类型
                types = info.find_all('a')
                if re.search('title="(.*?)"', str(types[3])):
                    user_type = (re.search('title="(.*?)"', str(types[3]))).group(1)
                else:
                    user_type = '普通用户'
                data.append(''.join(user_type))
                
                #内容
                content = info.find_all('p')
                if len(content)>2:
                    if re.search(' 收起全文d', content[1].text):
                        full_content = content[1].text.strip().strip(' 收起全文d')
                    else:
                        full_content = content[0].text.strip()
                else:
                    full_content = content[0].text.strip()
                data.append(''.join(full_content))
                
                #转评赞
                rec = record.find_all('li')
                if rec[1].text.strip().lstrip('转发 '):
                    data.append(int(rec[1].text.strip().lstrip('转发 ')))
                else:
                    rec_zhuan = 0
                    data.append(rec_zhuan)
                if rec[2].text.lstrip('评论 '):
                    data.append(int(rec[2].text.lstrip('评论 ')))
                else:
                    rec_ping = 0
                    data.append(rec_ping)
                if rec[3].text.strip():
                    data.append(int(rec[3].text.strip()))
                else:
                    rec_zan = 0
                    data.append(rec_zan)
                
        except: 
            print('获取数据失败', date, page)
            return 0
    return data

def save_data_to_csv(data):
    data = np.array(data).reshape(-1,10)
    result_weibo = pd.DataFrame(data)
    result_weibo.to_csv(data_file_name, mode = 'a', index = False, encoding = 'gb18030', header = False)
    

data_file_name = './2014年8月csv/20140831_0-0901_0.csv'
column = pd.DataFrame({}, columns = ['日期','时间','用户名','用户ID', '博文ID', '用户类别','内容','转发','评论','点赞'])
column.to_csv(data_file_name, index = False, encoding = 'gb18030')

if __name__ == '__main__':
    page = 1
    page_num = re.findall('第(.*?)页</a></li>', get_page_session(2019).text)
    pages = page_num[-1]
    print('共' + pages + '页')

    for page in range(int(pages)):
        print('-----------------正在爬取第' + str(page+1) + '页数据-----------------')
        save_data_to_csv(get_data(2019, page+1))

print('爬取' + data_file_name + '完成！')
winsound.Beep(500,1000)

F:\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
F:\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
F:\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
F:\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate ver

共23页
-----------------正在爬取第1页数据-----------------


F:\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


-----------------正在爬取第2页数据-----------------


F:\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


...

-----------------正在爬取第18页数据-----------------


F:\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


-----------------正在爬取第19页数据-----------------


F:\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


-----------------正在爬取第20页数据-----------------
